In [1]:
import pandas as pd
import pickle
import joblib
import numpy as np
import torch
# df = pd.read_pickle('poses/results/demo_TIGER_WOODS_THqn7IjaiEY_1c3e5a96-37ee-4fe4-8df9-1c97cea2cf67.pkl')

# print(df.head())

In [10]:
#x = joblib.load('poses/results/demo_TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8.pkl')
x = joblib.load('poses/results/demo_TIGER_WOODS_4HzLO88ryCU_a941d487-8693-40b6-8835-75b41c87b662.pkl')


#print(x['/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_THqn7IjaiEY_1c3e5a96-37ee-4fe4-8df9-1c97cea2cf67/img/000001.jpg']['smpl'])

In [11]:
x

{'/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_a941d487-8693-40b6-8835-75b41c87b662/img/000001.jpg': {'time': 0,
  'shot': 0,
  'frame_path': '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_a941d487-8693-40b6-8835-75b41c87b662/img/000001.jpg',
  'tracked_ids': [1],
  'tracked_bbox': [array([ 41.02694,  73.92018, 141.75671, 274.48267], dtype=float32)],
  'tid': [1],
  'bbox': [array([ 41.02694,  73.92018, 141.75671, 274.48267], dtype=float32)],
  'tracked_time': [0],
  'appe': [array([ 8.824416 ,  9.8371105,  6.9036374, ..., 11.109199 , 12.218206 ,
           8.754949 ], dtype=float32)],
  'loca': [array([ 0.462799  ,  0.3314514 ,  0.3715999 ,  0.29606164,  0.3640522 ,
           0.3537171 ,  0.3494718 ,  0.4869594 ,  0.35726583,  0.61020446,
           0.35008487,  0.3335861 ,  0.3287167 ,  0.46732348,  0.34362948,
           0.59064233,  0.20953727,  0.4747169 ,  0.19962084,  0.52244407

In [12]:
import plotly.graph_objects as go

def viz(isaac_joints_3d):

    BONE_CONNECTIONS = {
        'torso_to_head': [0, 1],
        'torso_to_lower_waist': [0, 2],
        'torso_to_right_upper_arm': [0, 3],
        'torso_to_left_upper_arm': [0, 4],
        'lower_waist_to_pelvis': [2, 5],
        'right_upper_arm_to_right_lower_arm': [3, 6],
        'left_upper_arm_to_left_lower_arm': [4, 7],
        'right_lower_arm_to_right_hand': [6, 10],
        'left_lower_arm_to_left_hand': [7, 11],
        'pelvis_to_right_thigh': [5, 8],
        'pelvis_to_left_thigh': [5, 9],
        'right_thigh_to_right_shin': [8, 12],
        'left_thigh_to_left_shin': [9, 13],
        'right_shin_to_right_foot': [12, 15],
        'left_shin_to_left_foot': [13, 16],  # Assuming Club is held in the right hand
    }

    # Extract the 3D joint coordinates
    print(isaac_joints_3d)
    joints_3d = isaac_joints_3d  # joints[0][:25]

    # Create a 3D scatter plot
    # print(joints_3d)
    # print(len(joints_3d))
    x_coords = [joint[0] for joint in joints_3d]
    y_coords = [joint[1] for joint in joints_3d]
    z_coords = [joint[2] for joint in joints_3d]
    # x_coords = [1, 2, 3]
    # y_coords = [4, 5, 6]
    # z_coords = [7, 8, 9]

    print(x_coords, y_coords, z_coords)
    fig = go.Figure(data=[go.Scatter3d(
        x=x_coords,
        y=y_coords,
        z=z_coords,
        mode='markers',
        marker=dict(
            size=5,
            color=['blue' if i == 0 else 'red' for i in range(len(joints_3d))],  # Color the 45th joint in blue
        )
    )])

    # # Add lines for each bone defined in BONE_CONNECTIONS
    for bone_name, (start_joint, end_joint) in BONE_CONNECTIONS.items():
        fig.add_trace(go.Scatter3d(
            x=[joints_3d[start_joint][0], joints_3d[end_joint][0]],
            y=[joints_3d[start_joint][1], joints_3d[end_joint][1]],
            z=[joints_3d[start_joint][2], joints_3d[end_joint][2]],
            mode='lines',
            line=dict(color='blue', width=2),
            name=bone_name
        ))



    # Set labels
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    # Return image of plot
    fig.write_image("plot_image.png")

In [14]:
angle_sequence = []
joint_positions = []
for i in range(1, 239):
    i_str = str(i).zfill(6)
    file_name = f'/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_a941d487-8693-40b6-8835-75b41c87b662/img/{i_str}.jpg'
    joints = x[file_name]['3d_joints']
    joints_3d = joints[0]

    new_joints_3d = compute_joint_positions(joints_3d)
    # new_joints_3d = np.array([
    #     joints_3d[1],  # OP Neck (torso)
    #     joints_3d[0],  # OP Nose (head)
    #     joints_3d[8],  # OP MidHip (lower_waist)
    #     joints_3d[2],  # OP RShoulder (right_upper_arm)
    #     joints_3d[5],  # OP LShoulder (left_upper_arm)
    #     joints_3d[40],  # OP MidHip (pelvis)
    #     joints_3d[3],  # OP RElbow (right_lower_arm)
    #     joints_3d[6],  # OP LElbow (left_lower_arm)
    #     joints_3d[9],  # OP RHip (right_thigh)
    #     joints_3d[12], # OP LHip (left_thigh)
    #     joints_3d[4],  # OP RWrist (right_hand)
    #     joints_3d[7],  # OP LWrist (left_hand)
    #     joints_3d[10], # OP RKnee (right_shin)
    #     joints_3d[13], # OP LKnee (left_shin)
    #     (0,0,0),          # Club (not available in JOINT_NAMES)
    #     joints_3d[11], # OP RAnkle (right_foot)
    #     joints_3d[14]  # OP LAnkle (left_foot)
    # ])

    #cur_angles=compute_joint_angles_batch(torch.tensor([new_joints_3d]))
    #angle_sequence.append(cur_angles[0].tolist())
    joint_positions.append(new_joints_3d.tolist())

    


import json

# Define the output file path
output_file_path = 'poses/results/joint_position_sequence.json'

# Write the angle_sequence to the output file
with open(output_file_path, 'w') as f:
    json.dump(joint_positions, f)

print(f"Angle sequence has been written to {output_file_path}")


NameError: name 'compute_joint_positions' is not defined

In [6]:
x.keys()

dict_keys(['/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000001.jpg', '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000002.jpg', '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000003.jpg', '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000004.jpg', '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000005.jpg', '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000006.jpg', '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOOD

In [23]:




joints = x['/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000060.jpg']['3d_joints']


KeyError: '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8/img/000060.jpg'

In [25]:
import plotly.graph_objects as go

BONE_CONNECTIONS = {
    'nose_to_neck': [0, 1],
    'neck_to_rshoulder': [1, 2],
    'rshoulder_to_relbow': [2, 3],
    'relbow_to_rwrist': [3, 4],
    'neck_to_lshoulder': [1, 5],
    'lshoulder_to_lelbow': [5, 6],
    'lelbow_to_lwrist': [6, 7],
    'neck_to_midhip': [1, 8],
    'midhip_to_rhip': [8, 9],
    'rhip_to_rknee': [9, 10],
    'rknee_to_rankle': [10, 11],
    'midhip_to_lhip': [8, 12],
    'lhip_to_lknee': [12, 13],
    'lknee_to_lankle': [13, 14],
    'nose_to_reye': [0, 15],
    'nose_to_leye': [0, 16],
    'reye_to_rear': [15, 17],
    'leye_to_lear': [16, 18],
    'lankle_to_lbigtoe': [14, 19],
    'lankle_to_lsmalltoe': [14, 20],
    'lankle_to_lheel': [14, 21],
    'rankle_to_rbigtoe': [11, 22],
    'rankle_to_rsmalltoe': [11, 23],
    'rankle_to_rheel': [11, 24],
}

# Extract the 3D joint coordinates
joints_3d = joints[0]
joints_3d[:, [0, 1, 2]] = joints_3d[:, [0,2,1]]  # Make the 3rd value be the first on the second axis
joints_3d[:, 2] *= -1  # Make the 3rd value negative
print("norm", np.linalg.norm(joints_3d[3] - joints_3d[4]))
print(joints_3d)
joints_3d *= (0.34/ np.linalg.norm(joints_3d[3] - joints_3d[4]))
print(joints_3d)
joints_3d -= joints_3d[14]
# #joints_3d[:, 2] = -joints_3d[:, 2]  # Make the 3rd value negative
# # Subtract left foot position (left ankle) from all values
# left_foot_position = joints_3d[14]  # left ankle joint index is 14
# joints_3d -= left_foot_position
print(joints_3d)
# Create a 3D scatter plot

# joints_3d = joint_positions[60]
x_coords = [joint[0] for joint in joints_3d]
y_coords = [joint[1] for joint in joints_3d]
z_coords = [joint[2] for joint in joints_3d]
# x_coords = [1, 2, 3]
# y_coords = [4, 5, 6]
# z_coords = [7, 8, 9]

fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=5,
        color=['blue' if i == 44 else 'red' for i in range(len(joints_3d))],  # Color the 45th joint in blue
    )
)])

# # Add lines for each bone defined in BONE_CONNECTIONS
# for bone_name, (start_joint, end_joint) in BONE_CONNECTIONS.items():
#     fig.add_trace(go.Scatter3d(
#         x=[joints_3d[start_joint][0], joints_3d[end_joint][0]],
#         y=[joints_3d[start_joint][1], joints_3d[end_joint][1]],
#         z=[joints_3d[start_joint][2], joints_3d[end_joint][2]],
#         mode='lines',
#         line=dict(color='blue', width=2),
#         name=bone_name
#     ))



# Set labels
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    )
)

# Show the plot
fig.show()


norm 0.33999994
[[ 0.26848775 -1.9823486   0.11944408]
 [ 0.28845304 -1.7910571  -0.04806028]
 [ 0.35597625 -1.722744    0.14976934]
 [ 0.06956729 -1.8452026   0.28324917]
 [-0.07476958 -2.0612595   0.06396347]
 [ 0.07993516 -1.760895   -0.15852635]
 [-0.25881198 -1.7828964  -0.08300663]
 [-0.20202614 -2.0857644   0.03954317]
 [ 0.10808711 -1.1528382   0.05227285]
 [ 0.18171665 -1.0309199   0.09491767]
 [ 0.25373212 -0.5330402   0.20001455]
 [ 0.35445046 -0.09248342 -0.08686364]
 [ 0.01658177 -1.035231    0.02606665]
 [-0.01212547 -0.5326561   0.04756379]
 [ 0.          0.         -0.        ]
 [ 0.33497375 -2.0118482   0.08139491]
 [ 0.2530166  -2.0318573   0.05624892]
 [ 0.4100978  -1.9498383  -0.01682814]
 [ 0.22859104 -1.9937559  -0.07249959]
 [-0.04984574  0.06803235  0.25540894]
 [-0.1234722   0.07680091  0.17339803]
 [ 0.00558535  0.04771809 -0.07648849]
 [ 0.31801605  0.08760091  0.10492148]
 [ 0.42092115  0.06330588  0.06220845]
 [ 0.38160437 -0.07548955 -0.18121083]
 [ 0.3767

In [46]:


new_joints_3d = np.array([
    joints_3d[1],  # OP Neck (torso)
    joints_3d[0],  # OP Nose (head)
    joints_3d[8],  # OP MidHip (lower_waist)
    joints_3d[2],  # OP RShoulder (right_upper_arm)
    joints_3d[5],  # OP LShoulder (left_upper_arm)
    joints_3d[40],  # OP MidHip (pelvis)
    joints_3d[3],  # OP RElbow (right_lower_arm)
    joints_3d[6],  # OP LElbow (left_lower_arm)
    joints_3d[9],  # OP RHip (right_thigh)
    joints_3d[12], # OP LHip (left_thigh)
    joints_3d[4],  # OP RWrist (right_hand)
    joints_3d[7],  # OP LWrist (left_hand)
    joints_3d[10], # OP RKnee (right_shin)
    joints_3d[13], # OP LKnee (left_shin)
    (0,0,0),          # Club (not available in JOINT_NAMES)
    joints_3d[11], # OP RAnkle (right_foot)
    joints_3d[14]  # OP LAnkle (left_foot)
])

# compute_joint_angles_batch(torch.tensor([new_joints_3d]))
new_joints_3d

IndexError: list index out of range

In [16]:
def compute_joint_positions(joints_3d):
    joints_3d[:, [0, 1, 2]] = joints_3d[:, [0,2,1]]  # Make the 3rd value be the first on the second axis
    joints_3d[:, 2] *= -1  # Make the 3rd value negative
    joints_3d *= (0.34 / np.linalg.norm(joints_3d[3] - joints_3d[4]))
    joints_3d -= joints_3d[14]

    new_joints_3d = np.array([
        joints_3d[1],  # OP Neck (torso)
        joints_3d[0],  # OP Nose (head)
        joints_3d[8],  # OP MidHip (lower_waist)
        joints_3d[2],  # OP RShoulder (right_upper_arm)
        joints_3d[5],  # OP LShoulder (left_upper_arm)
        joints_3d[40],  # OP MidHip (pelvis)
        joints_3d[3],  # OP RElbow (right_lower_arm)
        joints_3d[6],  # OP LElbow (left_lower_arm)
        joints_3d[9],  # OP RHip (right_thigh)
        joints_3d[12], # OP LHip (left_thigh)
        joints_3d[4],  # OP RWrist (right_hand)
        joints_3d[7],  # OP LWrist (left_hand)
        joints_3d[10], # OP RKnee (right_shin)
        joints_3d[13], # OP LKnee (left_shin)
        (0,0,0),          # Club (not available in JOINT_NAMES)
        joints_3d[11], # OP RAnkle (right_foot)
        joints_3d[14]  # OP LAnkle (left_foot)
    ])

    return new_joints_3d

In [48]:
JOINT_NAMES = [
# 25 OpenPose joints (in the order provided by OpenPose)
'OP Nose',
'OP Neck',
'OP RShoulder',
'OP RElbow',
'OP RWrist',
'OP LShoulder',
'OP LElbow',
'OP LWrist',
'OP MidHip',
'OP RHip',
'OP RKnee',
'OP RAnkle',
'OP LHip',
'OP LKnee',
'OP LAnkle',
'OP REye',
'OP LEye',
'OP REar',
'OP LEar',
'OP LBigToe',
'OP LSmallToe',
'OP LHeel',
'OP RBigToe',
'OP RSmallToe',
'OP RHeel',
# 24 Ground Truth joints (superset of joints from different datasets)
'Right Ankle',
'Right Knee',
'Right Hip',
'Left Hip',
'Left Knee',
'Left Ankle',
'Right Wrist',
'Right Elbow',
'Right Shoulder',
'Left Shoulder',
'Left Elbow',
'Left Wrist',
'Neck (LSP)',
'Top of Head (LSP)',
'Pelvis (MPII)',
'Thorax (MPII)',
'Spine (H36M)',
'Jaw (H36M)',
'Head (H36M)',
'ROOT'
]
print(len(JOINT_NAMES))

45


In [88]:
import numpy as np

def compute_all_joint_angles(pose_array):
    """
    Compute all joint angles from OpenPose joint positions to match Isaac Sim format
    
    Args:
        pose_array: numpy array of shape (45, 3) containing joint positions
        
    Returns:
        numpy array of shape (21,) matching initial_positions format
    """
    # OpenPose indices
    NECK_IDX = 1        # OP Neck
    R_SHOULDER_IDX = 2  # OP RShoulder
    R_ELBOW_IDX = 3    # OP RElbow
    R_WRIST_IDX = 4    # OP RWrist
    L_SHOULDER_IDX = 5  # OP LShoulder
    L_ELBOW_IDX = 6    # OP LElbow
    L_WRIST_IDX = 7    # OP LWrist
    MID_HIP_IDX = 8    # OP MidHip
    R_HIP_IDX = 9      # OP RHip
    R_KNEE_IDX = 10    # OP RKnee
    R_ANKLE_IDX = 11   # OP RAnkle
    L_HIP_IDX = 12     # OP LHip
    L_KNEE_IDX = 13    # OP LKnee
    L_ANKLE_IDX = 14   # OP LAnkle
    R_FOOT_IDX = 22    # OP RBigToe
    L_FOOT_IDX = 19    # OP LBigToe
    
    def compute_waist_angles(neck, mid_hip):
        # Calculate waist bend forward/backward and side-to-side
        spine_vector = neck - mid_hip
        spine_vector = spine_vector / np.linalg.norm(spine_vector)
        
        # Forward/backward bend (in sagittal plane)
        sagittal_angle = np.arctan2(spine_vector[2], spine_vector[1])
        
        # Side-to-side bend (in coronal plane)
        coronal_angle = np.arctan2(spine_vector[0], spine_vector[1])
        
        return sagittal_angle / np.pi, coronal_angle / np.pi
    
    def compute_arm_angles(shoulder_idx, elbow_idx, wrist_idx):
        shoulder_pos = pose_array[shoulder_idx]
        elbow_pos = pose_array[elbow_idx]
        wrist_pos = pose_array[wrist_idx]
        
        spine = pose_array[NECK_IDX] - pose_array[MID_HIP_IDX]
        shoulder_line = pose_array[R_SHOULDER_IDX] - pose_array[L_SHOULDER_IDX]
        coronal_normal = np.cross(spine, shoulder_line)
        coronal_normal = coronal_normal / np.linalg.norm(coronal_normal)
        sagittal_normal = np.cross(coronal_normal, spine)
        # Upper arm angles
        upper_arm = elbow_pos - shoulder_pos
        upper_arm = upper_arm / np.linalg.norm(upper_arm)
        
        # Abduction angle relative to coronal plane
        abduction = np.arccos(np.dot(upper_arm, coronal_normal))
        if np.dot(upper_arm, shoulder_line) < 0:  # Check if arm is moving away from body
            abduction = -abduction
            
        # Flexion angle
        flexion = np.arccos(np.dot(upper_arm, sagittal_normal))
        # sagittal_projection = np.array([upper_arm[2], upper_arm[1]])
        # sagittal_projection = sagittal_projection / np.linalg.norm(sagittal_projection)
        # flexion = np.arccos(np.dot(sagittal_projection, [0, 1]))
        # if upper_arm[2] < 0:
        #     flexion = -flexion
            
        # Lower arm angle (elbow)
        forearm = wrist_pos - elbow_pos
        upper_arm = elbow_pos - shoulder_pos
        elbow_angle = np.arccos(np.dot(forearm, upper_arm) / 
                               (np.linalg.norm(forearm) * np.linalg.norm(upper_arm)))
        
        return abduction, flexion, (elbow_angle)
    
    def compute_leg_angles(hip_idx, knee_idx, ankle_idx, foot_idx):
        hip_pos = pose_array[hip_idx]
        knee_pos = pose_array[knee_idx]
        ankle_pos = pose_array[ankle_idx]
        foot_pos = pose_array[foot_idx]
        
        # Thigh angles
        thigh = knee_pos - hip_pos
        thigh = thigh / np.linalg.norm(thigh)
        
        # Calculate three rotation angles for thigh
        thigh_x = np.arctan2(thigh[2], thigh[1])  # forward/backward
        thigh_y = np.arctan2(thigh[0], thigh[1])  # side-to-side
        thigh_z = np.arctan2(thigh[0], thigh[2])  # rotation
        
        # Knee angle
        shank = ankle_pos - knee_pos
        knee_angle = np.arccos(np.dot(thigh, shank) /
                             (np.linalg.norm(thigh) * np.linalg.norm(shank)))
        
        # Foot angles
        foot = foot_pos - ankle_pos
        foot_pitch = np.arctan2(foot[2], foot[1])  # up/down
        foot_roll = np.arctan2(foot[0], foot[1])   # inversion/eversion
        
        return (thigh_x / np.pi, thigh_y / np.pi, thigh_z / np.pi, 
                knee_angle / np.pi, foot_pitch / np.pi, foot_roll / np.pi)
    
    # Calculate all angles
    waist_sagittal, waist_coronal = compute_waist_angles(
        pose_array[NECK_IDX], pose_array[MID_HIP_IDX])
    
    r_abd, r_flex, r_elbow = compute_arm_angles(
        R_SHOULDER_IDX, R_ELBOW_IDX, R_WRIST_IDX)
    l_abd, l_flex, l_elbow = compute_arm_angles(
        L_SHOULDER_IDX, L_ELBOW_IDX, L_WRIST_IDX)
    
    pelvis_angle = np.arctan2(
        pose_array[R_HIP_IDX][0] - pose_array[L_HIP_IDX][0],
        pose_array[R_HIP_IDX][2] - pose_array[L_HIP_IDX][2]) / np.pi
    
    r_thigh_x, r_thigh_y, r_thigh_z, r_knee, r_foot_pitch, r_foot_roll = compute_leg_angles(
        R_HIP_IDX, R_KNEE_IDX, R_ANKLE_IDX, R_FOOT_IDX)
    l_thigh_x, l_thigh_y, l_thigh_z, l_knee, l_foot_pitch, l_foot_roll = compute_leg_angles(
        L_HIP_IDX, L_KNEE_IDX, L_ANKLE_IDX, L_FOOT_IDX)
    
    # Assemble in the same order as initial_positions
    return np.array([
        waist_coronal,  # lower_waist
        waist_sagittal,   # lower_waist
        r_abd,          # right_upper_arm
        r_flex,         # right_upper_arm
        l_abd,          # left_upper_arm
        l_flex,         # left_upper_arm
        pelvis_angle,   # pelvis
        r_elbow,        # right_lower_arm
        l_elbow,        # left_lower_arm
        r_thigh_x,      # right_thigh: x
        r_thigh_y,      # right_thigh: y
        r_thigh_z,      # right_thigh: z
        l_thigh_x,      # left_thigh: x
        l_thigh_y,      # left_thigh: y
        l_thigh_z,      # left_thigh: z
        r_knee,         # right_knee
        l_knee,         # left_knee
        r_foot_pitch,   # right_foot
        r_foot_roll,    # right_foot
        l_foot_pitch,   # left_foot
        l_foot_roll,    # left_foot
    ])

# Example usage:
# pose_data = np.random.rand(45, 3)  # Your pose data here
# joint_angles = compute_all_joint_angles(pose_data)

In [89]:
joint_angles = compute_all_joint_angles(joints_3d)
print(joint_angles)

[ 0.5578012   0.5518533   0.9407179   1.5012295  -0.9618227   1.6079078
 -0.709464    0.3826844   0.5346749  -0.5371449   0.6105247   0.90028393
 -0.44429156  0.33732066  0.9027711   0.14125776  0.12458071 -0.57734203
  0.5320902  -0.46259755  0.48720014]


In [36]:
import numpy as np
# isaac_joint_ex = np.array([[[-6.3307e-02,  4.1133e-02,  1.0862e+00,  8.2588e-01,  4.2306e-02,
#            5.6173e-01, -2.4218e-02, -3.2339e-01,  1.3146e-01, -1.0134e-01,
#            1.8591e-01,  5.0460e-01, -2.2101e-02],
#          [ 1.1259e-01,  2.2687e-02,  1.1556e+00,  8.2588e-01,  4.2306e-02,
#            5.6173e-01, -2.4218e-02, -2.8213e-01,  1.1121e-01, -2.1674e-01,
#            1.8591e-01,  5.0460e-01, -2.2101e-02],
#          [-2.5426e-01,  6.1176e-02,  9.5980e-01,  9.9602e-01,  1.2835e-02,
#            5.0397e-02,  7.2358e-02, -3.5577e-01,  1.5930e-01, -2.6902e-02,
#            2.4795e-01,  1.2763e-01,  4.1328e-02],
#          [-2.2640e-02, -1.3388e-01,  1.1008e+00,  8.2717e-01,  3.5315e-01,
#            4.3171e-01,  6.8515e-02, -4.1392e-01,  1.5754e-01, -1.5356e-01,
#            2.4475e-01,  7.4676e-01, -4.8113e-03],
#          [ 7.1208e-03,  2.0450e-01,  1.1153e+00,  8.1802e-01, -1.3191e-01,
#            5.5824e-01, -4.2620e-02, -2.5521e-01,  1.2960e-01, -1.4447e-01,
#           -2.5006e-02, -2.7023e-01, -2.3035e-01],
#          [-2.6974e-01,  5.4893e-02,  7.9583e-01,  9.9563e-01, -3.3860e-02,
#            4.4935e-02,  7.4529e-02, -3.6865e-01,  1.4968e-01, -2.1715e-02,
#           -2.3476e-01,  5.6380e-02,  8.7674e-02],
#          [-8.3173e-02, -9.7493e-02,  7.9715e-01,  6.6653e-01,  1.9141e-01,
#            7.1758e-01, -6.4605e-02, -6.0180e-01,  2.1870e-01, -1.9162e-01,
#            2.4307e-01,  7.5047e-01, -4.0323e-03],
#          [-1.0610e-01,  3.0824e-01,  8.4400e-01,  6.5039e-01,  1.6765e-02,
#            7.4971e-01,  1.2101e-01, -1.8250e-01,  1.1743e-01, -1.3805e-01,
#           -2.5836e-02, -2.7134e-01, -2.3043e-01],
#          [-2.5797e-01, -4.6732e-02,  7.6215e-01,  9.6755e-01, -1.0398e-01,
#           -2.1086e-01,  9.2655e-02, -3.2300e-01,  1.2735e-01,  2.0599e-02,
#           -1.7400e-01, -2.3558e-01,  1.3588e-01],
#          [-2.8826e-01,  1.5059e-01,  7.5001e-01,  9.5321e-01,  8.0019e-03,
#           -2.8794e-01,  9.1777e-02, -3.9120e-01,  8.7349e-02, -5.4783e-02,
#           -2.6924e-01,  8.5775e-02, -1.3949e-01],
#          [ 1.4263e-01,  3.9141e-02,  6.3119e-01,  6.6653e-01,  1.9141e-01,
#            7.1758e-01, -6.4605e-02, -6.6380e-01,  2.3842e-01, -2.5975e-01,
#            2.4307e-01,  7.5047e-01, -4.0323e-03],
#          [ 6.6365e-02,  1.9521e-01,  6.1016e-01,  6.5039e-01,  1.6765e-02,
#            7.4971e-01,  1.2101e-01, -1.6380e-01,  9.4535e-02, -1.1319e-01,
#           -2.5836e-02, -2.7134e-01, -2.3043e-01],
#          [-9.9379e-02, -1.0447e-01,  4.0015e-01,  9.8507e-01, -1.2768e-01,
#            1.0125e-01,  5.5628e-02, -1.4282e-01,  8.1667e-02,  2.8256e-02,
#           -1.0259e-01, -7.4234e-01,  2.6256e-01],
#          [-7.2986e-02,  1.6679e-01,  4.1131e-01,  9.9121e-01, -1.0198e-02,
#           -9.4919e-02,  9.1559e-02, -2.0629e-01,  1.4250e-02, -2.8939e-02,
#           -3.5942e-02, -1.1914e+00, -9.0643e-02],
#          [ 8.5325e-02, -4.5011e-02,  9.3270e-01,  6.5039e-01,  1.6765e-02,
#            7.4971e-01,  1.2101e-01, -4.6299e-02, -1.6010e-02,  3.8042e-03,
#           -2.5836e-02, -2.7134e-01, -2.3043e-01],
#          [-1.5227e-01, -2.0182e-01,  2.8334e-02,  9.9016e-01, -1.0201e-01,
#           -1.9005e-02,  9.3868e-02, -4.7292e-03,  1.7406e-02,  2.8634e-02,
#           -5.2376e-01, -2.0127e-01, -3.3221e-02],
#          [-1.4144e-02,  1.6601e-01,  2.6956e-02,  9.9554e-01,  6.6397e-04,
#            3.7092e-04,  9.4339e-02, -5.3240e-03,  7.7487e-04,  1.1875e-02,
#           -6.1000e-02, -1.7204e-01, -5.9444e-03]]])
isaac_joint_ex = np.array([[ 0.8412,  0.3843,  0.8305],
         [ 0.9874,  0.4583,  0.9266],
         [ 0.6409,  0.2815,  0.7014],
         [ 0.9433,  0.2511,  0.8962],
         [ 0.8314,  0.5642,  0.8254],
         [ 0.5277,  0.2133,  0.6026],
         [ 0.9829,  0.0236,  0.6868],
         [ 0.7637,  0.6226,  0.5267],
         [ 0.5232,  0.1064,  0.6149],
         [ 0.4776,  0.2870,  0.5421],
         [ 1.1455,  0.2771,  0.6062],
         [ 1.0644,  0.5402,  0.5294],
         [ 0.2965, -0.0208,  0.3068],
         [ 0.2137,  0.1272,  0.2827],
         [ 0.8685,  0.3817,  0.8435],
         [ 0.1285, -0.1416, -0.0124],
         [ 0.0000,  0.0000,  0.0000]])

In [38]:
isaac_joints_3d = isaac_joint_ex #isaac_joint_ex[0, :, :3]
isaac_joints_3d.shape

(17, 3)

In [39]:
import plotly.graph_objects as go

BONE_CONNECTIONS = {
    'torso_to_head': [0, 1],
    'torso_to_lower_waist': [0, 2],
    'torso_to_right_upper_arm': [0, 3],
    'torso_to_left_upper_arm': [0, 4],
    'lower_waist_to_pelvis': [2, 5],
    'right_upper_arm_to_right_lower_arm': [3, 6],
    'left_upper_arm_to_left_lower_arm': [4, 7],
    'right_lower_arm_to_right_hand': [6, 10],
    'left_lower_arm_to_left_hand': [7, 11],
    'pelvis_to_right_thigh': [5, 8],
    'pelvis_to_left_thigh': [5, 9],
    'right_thigh_to_right_shin': [8, 12],
    'left_thigh_to_left_shin': [9, 13],
    'right_shin_to_right_foot': [12, 15],
    'left_shin_to_left_foot': [13, 16],  # Assuming Club is held in the right hand
}

# Extract the 3D joint coordinates
joints_3d = isaac_joints_3d  # joints[0][:25]
joints_3d /= np.linalg.norm(joints_3d[3] - joints_3d[4])
# Create a 3D scatter plot
# print(joints_3d)
# print(len(joints_3d))
x_coords = [joint[0] for joint in joints_3d]
y_coords = [joint[1] for joint in joints_3d]
z_coords = [joint[2] for joint in joints_3d]
# x_coords = [1, 2, 3]
# y_coords = [4, 5, 6]
# z_coords = [7, 8, 9]

fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=5,
        color=['blue' if i == 0 else 'red' for i in range(len(joints_3d))],  # Color the 45th joint in blue
    )
)])

# # Add lines for each bone defined in BONE_CONNECTIONS
for bone_name, (start_joint, end_joint) in BONE_CONNECTIONS.items():
    fig.add_trace(go.Scatter3d(
        x=[joints_3d[start_joint][0], joints_3d[end_joint][0]],
        y=[joints_3d[start_joint][1], joints_3d[end_joint][1]],
        z=[joints_3d[start_joint][2], joints_3d[end_joint][2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name=bone_name
    ))



# Set labels
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Show the plot
fig.show()


In [37]:
isaac_joint_names = ['lower_waist', 'lower_waist', 'right_upper_arm', 'left_upper_arm', 'pelvis', 'right_lower_arm', 'left_lower_arm', 'right_thigh', 'left_thigh', 'right_shin', 'left_shin', 'right_foot', 'left_foot']


In [38]:
len(isaac_joint_names)

13

In [68]:
def get_waist_angles(neck, mid_hip, base, left_knee, right_knee):
    spine = neck - mid_hip
    vertical = base - mid_hip
    vertical = vertical / np.linalg.norm(vertical)
    horizontal = left_knee - right_knee
    horizontal = horizontal / np.linalg.norm(horizontal)

    # Normalize the length of the spine vector
    spine = spine / np.linalg.norm(spine)
    
    # Forward/backward bend (in sagittal plane)

    # Opposite over adjacent (z over y)
    forward_angle = np.arccos(np.dot(spine, vertical))
    side_angle = np.arccos(np.dot(spine, horizontal))
    
    return forward_angle, side_angle

def get_arm_angles(shoulder, elbow, wrist, neck, mid_hip):
    # Shoulder angle is dependent on the rotation of the spine and hips
        left_shoulder_abduction, left_shoulder_flexion,

    # Here we formulate shoulder angle in terms of abduction 
    # (moving away from sagital plane) and flexion (moving away from coronal plane)
    spine = neck - mid_hip
    shoulder_line = shoulder - elbow
    coronal_normal = np.cross(spine, shoulder_line)
    coronal_normal = coronal_normal / np.linalg.norm(coronal_normal)
    sagittal_normal = np.cross(coronal_normal, spine)

    # Shoulder angles
    upper_arm = elbow - shoulder
    upper_arm = upper_arm / np.linalg.norm(upper_arm)
    
    
    shoulder_abduction = np.arccos(np.dot(upper_arm, coronal_normal))
    shoulder_flexion = np.arccos(np.dot(upper_arm, sagittal_normal))

    # Elbow angle
    forearm = wrist - elbow
    forearm = forearm / np.linalg.norm(forearm)
    upper_arm = elbow - shoulder
    upper_arm = upper_arm / np.linalg.norm(upper_arm)
    elbow_angle = np.arccos(np.dot(forearm, upper_arm))
    
    return shoulder_abduction, shoulder_flexion, elbow_angle

def get_leg_angles(hip, knee, ankle, mid_hip, base, left_knee, right_knee):
    vertical = base - mid_hip
    vertical = vertical / np.linalg.norm(vertical)
    horizontal = left_knee - right_knee
    horizontal = horizontal / np.linalg.norm(horizontal)

    # Thigh angles
    thigh = knee - hip
    thigh = thigh / np.linalg.norm(thigh)

    # Hip angles are simply the angle on each 3D rotation axis
    thigh_forward = np.arccos(np.dot(thigh, vertical))
    thigh_side = np.arccos(np.dot(thigh, horizontal))
    # thigh_forward = np.arctan2(thigh[2], thigh[1])
    # thigh_out = np.arctan2(thigh[0], thigh[1]) 
    # thigh_twist = np.arctan2(thigh[0], thigh[2])  

    # Knee angle is the angle between the thigh and shin
    shin = ankle - knee
    shin = shin / np.linalg.norm(shin)
    knee_angle = np.arccos(np.dot(vertical, shin))

    # Foot tracking is not accurate enough, so we don't compute foot angles

    return thigh_forward, thigh_side, knee_angle
    

def compute_joint_angles(pose_array):
    '''
    Assume pose_array is in the following format:

    ['torso', 'head', 'lower_waist', 'right_upper_arm', 'left_upper_arm', 
    'pelvis', 'right_lower_arm', 'left_lower_arm', 'right_thigh', 
    'left_thigh', 'right_hand', 'left_hand', 'right_shin', 'left_shin', 
    'Club', 'right_foot', 'left_foot']
    '''

    # Center the pose at the left foot
    pose_array = pose_array - pose_array[16]

    
    waist_forwards, waist_sideways = get_waist_angles(pose_array[0], pose_array[2], (pose_array[16] + pose_array[15]) / 2, pose_array[12], pose_array[13])
    right_shoulder_abduction, right_shoulder_flexion, right_elbow_angle = get_arm_angles(pose_array[3], pose_array[6], pose_array[10], pose_array[1], pose_array[0])
    left_shoulder_abduction, left_shoulder_flexion, left_elbow_angle = get_arm_angles(pose_array[4], pose_array[7], pose_array[11], pose_array[1], pose_array[0])
    right_thigh_forward, right_thigh_side, right_knee_angle = get_leg_angles(pose_array[8], pose_array[12], pose_array[15], pose_array[2], (pose_array[16] + pose_array[15]) / 2, pose_array[12], pose_array[13])
    left_thigh_forward, left_thigh_side, left_knee_angle = get_leg_angles(pose_array[9], pose_array[13], pose_array[16], pose_array[2], (pose_array[16] + pose_array[15]) / 2, pose_array[12], pose_array[13])
    print(waist_forwards.shape)
    print(waist_sideways.shape)
    return np.array([
        waist_forwards, 
        waist_sideways, 
        right_shoulder_abduction, 
        right_shoulder_flexion, 
        left_shoulder_abduction, 
        left_shoulder_flexion, 
        right_thigh_forward, 
        right_thigh_side,  
        left_thigh_forward, 
        left_thigh_side, 
        right_knee_angle, 
        left_knee_angle
        ])

isaac_joint_angles = compute_joint_angles(isaac_joints_3d)
print(isaac_joint_angles)

()
()
[1.95694082 1.57877138 1.57079633 1.7305368  1.57079633 1.70939749
 0.23804266 1.42789031 0.40374151 1.62829758 0.36055553 0.08644649]


In [55]:
array([1.69021628, 1.69101144, 1.57079633, 1.76305979, 1.57079633,
       1.76507196, 0.30948977, 1.32033122, 0.30396942, 1.77190144,
       0.18234811, 0.20163746])

NameError: name 'array' is not defined

In [70]:
def get_waist_angles_batch(necks, mid_hips, bases, left_knees, right_knees):
    spines = necks - mid_hips
    verticals = bases - mid_hips
    verticals = verticals / np.linalg.norm(verticals, axis=1)[:, np.newaxis]
    horizontals = left_knees - right_knees
    horizontals = horizontals / np.linalg.norm(horizontals, axis=1)[:, np.newaxis]

    # Normalize the length of the spine vector
    spines = spines / np.linalg.norm(spines, axis=1)[:, np.newaxis]
    
    # Forward/backward bend relative to vertical and horizontals
    
    forward_angles = np.arccos(np.sum(spines * verticals, axis=1))
    side_angles = np.arccos(np.sum(spines * horizontals, axis=1))
    
    return forward_angles, side_angles

def get_arm_angles_batch(shoulders, elbows, wrists, necks, mid_hips):
    spines = necks - mid_hips
    shoulder_lines = shoulders - elbows
    
    # Calculate normals for each pose
    coronal_normals = np.cross(spines, shoulder_lines)
    coronal_normals = coronal_normals / np.linalg.norm(coronal_normals, axis=1)[:, np.newaxis]
    sagittal_normals = np.cross(coronal_normals, spines)

    upper_arms = elbows - shoulders
    upper_arms = upper_arms / np.linalg.norm(upper_arms, axis=1)[:, np.newaxis]
    
    shoulder_abductions = np.arccos(np.sum(upper_arms * coronal_normals, axis=1))
    shoulder_flexions = np.arccos(np.sum(upper_arms * sagittal_normals, axis=1))

    forearms = wrists - elbows
    forearms = forearms / np.linalg.norm(forearms, axis=1)[:, np.newaxis]
    upper_arms = elbows - shoulders
    upper_arms = upper_arms / np.linalg.norm(upper_arms, axis=1)[:, np.newaxis]
    elbow_angles = np.arccos(np.sum(forearms * upper_arms, axis=1))
    
    return shoulder_abductions, shoulder_flexions, elbow_angles

def get_leg_angles_batch(hips, knees, ankles, mid_hips, bases, left_knees, right_knees):
    verticals = bases - mid_hips
    verticals = verticals / np.linalg.norm(verticals, axis=1)[:, np.newaxis]
    horizontals = left_knees - right_knees
    horizontals = horizontals / np.linalg.norm(horizontals, axis=1)[:, np.newaxis]

    thighs = knees - hips
    thighs = thighs / np.linalg.norm(thighs, axis=1)[:, np.newaxis]

    thigh_forwards = np.arccos(np.sum(thighs * verticals, axis=1))
    thigh_sides = np.arccos(np.sum(thighs * horizontals, axis=1))

    shins = ankles - knees
    shins = shins / np.linalg.norm(shins, axis=1)[:, np.newaxis]
    knee_angles = np.arccos(np.sum(verticals * shins, axis=1))

    return thigh_forwards, thigh_sides, knee_angles

def compute_joint_angles_batch(pose_arrays):
    """
    Compute joint angles for multiple poses at once
    
    Args:
        pose_arrays: numpy array of shape (N, 17, 3) where N is number of poses
        
    Returns:
        numpy array of shape (N, 12) containing computed angles for each pose
    """
    # Center each pose at its left foot
    pose_arrays = pose_arrays - pose_arrays[:, 16:17]

    # Calculate base points (midpoint of feet) for each pose
    bases = (pose_arrays[:, 16] + pose_arrays[:, 15]) / 2

    # Calculate all angles in batches
    waist_forwards, waist_sideways = get_waist_angles_batch(
        pose_arrays[:, 1], pose_arrays[:, 0], bases, 
        pose_arrays[:, 12], pose_arrays[:, 13]
    )
    
    right_shoulder_abduction, right_shoulder_flexion, right_elbow_angle = get_arm_angles_batch(
        pose_arrays[:, 3], pose_arrays[:, 6], pose_arrays[:, 10],
        pose_arrays[:, 1], pose_arrays[:, 0]
    )
    
    left_shoulder_abduction, left_shoulder_flexion, left_elbow_angle = get_arm_angles_batch(
        pose_arrays[:, 4], pose_arrays[:, 7], pose_arrays[:, 11],
        pose_arrays[:, 1], pose_arrays[:, 0]
    )
    
    right_thigh_forward, right_thigh_side, right_knee_angle = get_leg_angles_batch(
        pose_arrays[:, 8], pose_arrays[:, 12], pose_arrays[:, 15],
        pose_arrays[:, 2], bases, pose_arrays[:, 12], pose_arrays[:, 13]
    )
    
    left_thigh_forward, left_thigh_side, left_knee_angle = get_leg_angles_batch(
        pose_arrays[:, 9], pose_arrays[:, 13], pose_arrays[:, 16],
        pose_arrays[:, 2], bases, pose_arrays[:, 12], pose_arrays[:, 13]
    )

    # Stack all angles into final output array
    return np.stack([
        waist_forwards, waist_sideways,
        right_shoulder_abduction, right_shoulder_flexion,
        left_shoulder_abduction, left_shoulder_flexion,
        right_thigh_forward, right_thigh_side,
        left_thigh_forward, left_thigh_side,
        right_knee_angle, left_knee_angle
    ], axis=1)

# Example usage:
# poses = np.random.rand(32, 17, 3)  # 32 poses, each with 17 joints and 3 coordinates
# joint_angles = compute_joint_angles_batch(poses)  # Returns array of shape (32, 12)

isaac_joint_angles = compute_joint_angles_batch(np.array([isaac_joints_3d, isaac_joints_3d]))
print(isaac_joint_angles)

[[1.95694082 1.57877138 1.57079633 1.7305368  1.57079633 1.70939749
  0.23804266 1.42789031 0.40374151 1.62829758 0.36055553 0.08644649]
 [1.95694082 1.57877138 1.57079633 1.7305368  1.57079633 1.70939749
  0.23804266 1.42789031 0.40374151 1.62829758 0.36055553 0.08644649]]


In [115]:
def get_waist_angles_batch(necks, mid_hips, pelvis, bases, left_knees, right_knees):
    lower_spines = pelvis - mid_hips
    upper_spines = necks - mid_hips
    verticals = bases - mid_hips
    verticals = verticals / torch.norm(verticals, dim=1, keepdim=True)
    horizontals = left_knees - right_knees
    horizontals = horizontals / torch.norm(horizontals, dim=1, keepdim=True)

    # Normalize the length of the spine vector
    lower_spines = lower_spines / torch.norm(lower_spines, dim=1, keepdim=True)
    upper_spines = upper_spines / torch.norm(upper_spines, dim=1, keepdim=True)
    
    # Forward/backward bend relative to vertical and horizontals
    lower_waist_forward_angles = torch.acos(torch.sum(lower_spines * verticals, dim=1))
    lower_waist_side_angles = torch.acos(torch.sum(lower_spines * horizontals, dim=1))
    upper_spine_forward_angles = torch.acos(torch.sum(upper_spines * verticals, dim=1))
    upper_spine_side_angles = torch.acos(torch.sum(upper_spines * horizontals, dim=1))

    return lower_waist_forward_angles, lower_waist_side_angles, upper_spine_forward_angles, upper_spine_side_angles

def get_arm_angles_batch(shoulders, elbows, wrists, necks, mid_hips):
    # Shoulder angle is dependent on the rotation of the spine and hips

    # Here we formulate shoulder angle in terms of abduction 
    # (moving away from sagital plane) and flexion (moving away from coronal plane)
    spines = necks - mid_hips
    shoulder_lines = shoulders - elbows
    
    # Calculate normals for each pose
    coronal_normals = torch.cross(spines, shoulder_lines, dim=1)
    coronal_normals = coronal_normals / torch.norm(coronal_normals, dim=1, keepdim=True)
    sagittal_normals = torch.cross(coronal_normals, spines, dim=1)

    # Shoulder angles
    upper_arms = elbows - shoulders
    upper_arms = upper_arms / torch.norm(upper_arms, dim=1, keepdim=True)
    
    shoulder_abductions = torch.acos(torch.sum(upper_arms * coronal_normals, dim=1))
    shoulder_flexions = torch.acos(torch.sum(upper_arms * sagittal_normals, dim=1))

    # Elbow angle
    forearms = wrists - elbows
    forearms = forearms / torch.norm(forearms, dim=1, keepdim=True)
    upper_arms = elbows - shoulders
    upper_arms = upper_arms / torch.norm(upper_arms, dim=1, keepdim=True)
    elbow_angles = torch.acos(torch.sum(forearms * upper_arms, dim=1))
    
    return shoulder_abductions, shoulder_flexions, elbow_angles

def get_leg_angles_batch(hips, knees, ankles, mid_hips, bases, left_knees, right_knees):
    # Calculate vertical and horizontal vectors
    verticals = bases - mid_hips
    verticals = verticals / torch.norm(verticals, dim=1, keepdim=True)
    horizontals = left_knees - right_knees
    horizontals = horizontals / torch.norm(horizontals, dim=1, keepdim=True)
    
    # Hip angles are simply the angle on each 3D rotation axis
    thighs = knees - hips
    thighs = thighs / torch.norm(thighs, dim=1, keepdim=True)

    thigh_forwards = torch.acos(torch.sum(thighs * verticals, dim=1))
    thigh_sides = torch.acos(torch.sum(thighs * horizontals, dim=1))

    # Calculate knee angle between knee and vertical plane
    shins = ankles - knees
    shins = shins / torch.norm(shins, dim=1, keepdim=True)
    knee_angles = torch.acos(torch.sum(verticals * shins, dim=1))

    return thigh_forwards, thigh_sides, knee_angles

def compute_joint_angles_batch(pose_arrays):
    """
    Compute joint angles for multiple poses at once
    
    ['torso', 'head', 'lower_waist', 'right_upper_arm', 'left_upper_arm', 
    'pelvis', 'right_lower_arm', 'left_lower_arm', 'right_thigh', 
    'left_thigh', 'right_hand', 'left_hand', 'right_shin', 'left_shin', 
    'Club', 'right_foot', 'left_foot']
    """
    # Convert input to torch tensor if it isn't already
    if not isinstance(pose_arrays, torch.Tensor):
        pose_arrays = torch.tensor(pose_arrays, dtype=torch.float32)
        
    # Center each pose at its left foot
    pose_arrays = pose_arrays - pose_arrays[:, 16:17]

    # Calculate midpoint of feet
    bases = (pose_arrays[:, 16] + pose_arrays[:, 15]) / 2

    # Calculate all angles in batches
    lower_spine_forwards, lower_spine_sideways, upper_spine_forwards, upper_spine_sideways = get_waist_angles_batch(
        pose_arrays[:, 1], pose_arrays[:, 2], pose_arrays[:,5], bases, 
        pose_arrays[:, 12], pose_arrays[:, 13]
    )
    
    right_shoulder_abduction, right_shoulder_flexion, right_elbow_angle = get_arm_angles_batch(
        pose_arrays[:, 3], pose_arrays[:, 6], pose_arrays[:, 10],
        pose_arrays[:, 1], pose_arrays[:, 2]
    )
    
    left_shoulder_abduction, left_shoulder_flexion, left_elbow_angle = get_arm_angles_batch(
        pose_arrays[:, 4], pose_arrays[:, 7], pose_arrays[:, 11],
        pose_arrays[:, 1], pose_arrays[:, 2]
    )
    
    right_thigh_forward, right_thigh_side, right_knee_angle = get_leg_angles_batch(
        pose_arrays[:, 8], pose_arrays[:, 12], pose_arrays[:, 15],
        pose_arrays[:, 2], bases, pose_arrays[:, 12], pose_arrays[:, 13]
    )
    
    left_thigh_forward, left_thigh_side, left_knee_angle = get_leg_angles_batch(
        pose_arrays[:, 9], pose_arrays[:, 13], pose_arrays[:, 16],
        pose_arrays[:, 2], bases, pose_arrays[:, 12], pose_arrays[:, 13]
    )

    waist_line = pose_arrays[:, 8] - pose_arrays[:, 9]
    waist_line = waist_line / torch.norm(waist_line, dim=1, keepdim=True)

    shoulder_line = pose_arrays[:, 3] - pose_arrays[:, 4]
    shoulder_line = shoulder_line / torch.norm(shoulder_line, dim=1, keepdim=True)

    pelvis_angle = torch.acos(torch.sum(waist_line * shoulder_line, dim=1))

    # Stack all angles into final output tensor
    return torch.stack([
        lower_spine_forwards, lower_spine_sideways, upper_spine_forwards, upper_spine_sideways,
        right_shoulder_abduction, right_shoulder_flexion,
        left_shoulder_abduction, left_shoulder_flexion,
        right_thigh_forward, right_thigh_side,
        left_thigh_forward, left_thigh_side,
        right_knee_angle, left_knee_angle, pelvis_angle
    ], dim=1)

# Example usage:
# poses = torch.rand(32, 17, 3)  # 32 poses, each with 17 joints and 3 coordinates
# joint_angles = compute_joint_angles_batch(poses)  # Returns tensor of shape (32, 12)

isaac_joints_tensor = torch.tensor(isaac_joints_3d, dtype=torch.float32)
isaac_joint_angles = compute_joint_angles_batch(torch.stack([isaac_joints_tensor, isaac_joints_tensor]))
print(isaac_joint_angles)

tensor([[0.2788, 1.4830, 1.8691, 1.5833, 1.5708, 1.8982, 1.5708, 1.8468, 0.2380,
         1.4279, 0.4037, 1.6283, 0.3606, 0.0864, 0.2613],
        [0.2788, 1.4830, 1.8691, 1.5833, 1.5708, 1.8982, 1.5708, 1.8468, 0.2380,
         1.4279, 0.4037, 1.6283, 0.3606, 0.0864, 0.2613]])


In [75]:
compute_joint_angles_batch(torch.stack([torch.tensor(new_joints_3d), isaac_joints_tensor]))

tensor([[1.6902, 1.6910, 1.5708, 1.7631, 1.5708, 1.7651, 0.3095, 1.3203, 0.3040,
         1.7719, 0.1823, 0.2016],
        [1.9569, 1.5788, 1.5708, 1.7305, 1.5708, 1.7094, 0.2380, 1.4279, 0.4037,
         1.6283, 0.3606, 0.0864]], dtype=torch.float64)

In [25]:
sim= [1.7718, 1.4654, 1.8255, 1.5070, 1.5708, 1.9727, 1.5708, 1.8780, 0.2020, 1.6070, 0.3252, 1.7464, 0.1840, 0.3628, 0.3184]
ref= [2.4752, 1.7874, 2.1308, 1.9123, 1.5707, 2.0467, 1.5707, 2.1500, 0.3036, 1.6002, 0.3438, 1.7613, 0.1034, 0.3938, 0.6145]
wei=  [3.0,    1.0,    3.0,    1.0,    1.0,    1.0,    1.0,    1.0,    3.0,    3.0,    3.0,    3.0,    1.0,    1.0,    1.0]
joints =[
        "lower_spine_forwards", "lower_spine_sideways", "upper_spine_forwards", "upper_spine_sideways",
        "right_shoulder_abduction", "right_shoulder_flexion",
        "left_shoulder_abduction", "left_shoulder_flexion",
        "right_thigh_forward", "right_thigh_side",
        "left_thigh_forward", "left_thigh_side",
        "right_knee_angle", "left_knee_angle", "pelvis_angle"
    ]

import pandas as pd

# Combine lists into a DataFrame
data = {
    "joints": joints,
    "sim": sim,
    "ref": ref,
    "wei": wei
}

df = pd.DataFrame(data)
print(df)


                      joints     sim     ref  wei
0       lower_spine_forwards  1.7718  2.4752  3.0
1       lower_spine_sideways  1.4654  1.7874  1.0
2       upper_spine_forwards  1.8255  2.1308  3.0
3       upper_spine_sideways  1.5070  1.9123  1.0
4   right_shoulder_abduction  1.5708  1.5707  1.0
5     right_shoulder_flexion  1.9727  2.0467  1.0
6    left_shoulder_abduction  1.5708  1.5707  1.0
7      left_shoulder_flexion  1.8780  2.1500  1.0
8        right_thigh_forward  0.2020  0.3036  3.0
9           right_thigh_side  1.6070  1.6002  3.0
10        left_thigh_forward  0.3252  0.3438  3.0
11           left_thigh_side  1.7464  1.7613  3.0
12          right_knee_angle  0.1840  0.1034  1.0
13           left_knee_angle  0.3628  0.3938  1.0
14              pelvis_angle  0.3184  0.6145  1.0
